# News similarity modelling

[Dataset](https://www.kaggle.com/c/newssimil)

In [1]:
import pandas as pd
import csv
import numpy as np
import nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrgolubev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/andrgolubev/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
train_file = 'news_similarity_train.csv'

In [5]:
news_df = pd.read_csv(train_file, sep=",",
                      engine="c",
#                       names=['post_id','published_at','title','subtitle','content'],
#                       dtype={
#                           'post_id': np.int32,
#                           'published_at': np.datetime64,
#                           'title': str,
#                           'subtitle': str,
#                           'content': str
#                       }
#                       skiprows=skip
                     )

In [6]:
news_df.head()

,post_id,published_at,title,subtitle,content
0,1,2017-01-01T00:05:00Z,Полиция Стамбула собирается штурмовать ночной ...,"Согласно последним данным, в здании может нахо...",<p>Полиция Стамбула собирается штурмовать ночн...
1,2,2017-01-01T00:07:00Z,Расстрел посетителей в ночном клубе в Стамбуле...,"Ранее сообщалось, что полицейские собираются ш...","<p>Опубликовано видео, где злоумышленник расст..."
2,3,2017-01-01T00:21:00Z,Губернатор Стамбула: При нападении на ночной к...,"Ранее сообщалось, что от 40 до 60 человек полу...",<p>Губернатор Стамбула <span>Васип Шахин</span...
3,4,2017-01-01T00:29:00Z,Губернатор Стамбула назвал террористической ат...,"Согласно последним данным, погибло минимум 35 ...",<p>Губернатор Стамбула Васип Шахин назвал терр...
4,5,2017-01-01T00:47:00Z,Очевидцы: В ночной клуб в Стамбуле проходили б...,Ранее губернатор Стамбула Васип Шахин назвал п...,"<p>Посетители стамбульского клуба ""Рейна"", на ..."


In [7]:
import json
def get_json(myjson):
  try:
    return json.loads(myjson)
  except ValueError:
    return None

In [8]:
print(get_json(news_df['content'][103325]) is not None)
print(get_json(news_df['content'][0]) is not None)

True
False


In [9]:
j = get_json(news_df['content'][103297])
full_j = [row["value"]["blocks"][0]["text"] for row in j if row["type"] == "TEXT" or row["type"] == "SUBTITLE"]
"\n".join(full_j)

'21-летняя Райли Хемсон два года назад весила 115 кг\nДевушка занялась фитнесом и стала описывать всё, что с ней происходит, в соцсетях\nВо время своей работы парамедиком Хемсон сталкивалась с людьми, страдающими от избытка веса\nРайли делилась с подписчиками в "Инстаграме" не только своими успехами, но и переживаниями, ошибками\nОна начала активно пропагандировать здоровый образ жизни и идею о том, что надо любить своё тело и заботиться о нём на всех этапах жизни\nНе смотря на то что девушка является вегетарианкой, она набрала лишний вес из-за частого употребления пиццы и прочей вредной еды\nРайли вдохновила многих людей своими усилиями вести здоровый образ жизни. За год девушка потеряла 25 кг\nВ результате на страницу девушки в "Инстаграме" подписалось более 15 тыс. человек'

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(news_df['content'][16], 'html.parser')
soup.get_text().replace(u'\xa0', u' ')

'Президент США Барак Обама, генсек НАТО Йенс Столтенберг и глава европейской дипломатии Федерика Могерини выразили соболезнования в связи с терактом в Стамбуле.\n— Трагическое начало 2017 года в Стамбуле. Мои мысли сейчас с теми, кого затронуло это нападение в то время, когда они праздновали Новый год, — написал Йенс Столтенберг на своей страничке в "Твиттере".\n \nБарак Обама, в свою очередь, распорядился немедленно оказать всю необходимую помощь Турции после случившегося теракта в Стамбуле.\nФедерика Могерини также сказала, что этот год начался с трагических событий в Турции.\n— 2017 год начался с атаки на Стамбул. Наши мысли с погибшими и их родными. Мы продолжим работать над предотвращением таких трагедий, — написала Могерини в своём "Твиттере".\n \nНапомним, что в результате вооружённого нападения на элитный ночной клуб "Рейна" в Стамбуле погибли не менее 35 человек. За последними новостями по этой теме следите в нашей текстовой трансляции.'

In [11]:
news_df['content'][0]

'<p>Полиция Стамбула собирается штурмовать ночной клуб, где произошло нападение. Согласно последним данным, минимум один злоумышленник находится в здании. Ранено, по разным данным, от 40 до 60 человек.</p>\n<p>Жертвами стали не менее четырёх человек. Среди погибших — один полицейский.</p>\n<p>Во время празднеств\xa0в клубе "Рейна" находилось до 800 человек. Неизвестные произвели множество выстрелов и, как сообщается, взорвали две гранаты.</p>'

In [12]:
from pymystem3 import Mystem
m = Mystem()

In [13]:
lemmas = m.lemmatize(news_df['title'][0])

In [14]:
print(lemmas)

['полиция', ' ', 'стамбул', ' ', 'собираться', ' ', 'штурмовать', ' ', 'ночной', ' ', 'клуб', ', ', 'где', ' ', 'происходить', ' ', 'нападение', '\n']


In [15]:
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))
print(stop_words)

{'за', 'моя', 'ни', 'был', 'когда', 'тебя', 'этом', 'после', 'всего', 'мне', 'свою', 'три', 'у', 'или', 'него', 'такой', 'нибудь', 'много', 'от', 'не', 'чуть', 'этого', 'впрочем', 'два', 'них', 'бы', 'опять', 'здесь', 'их', 'с', 'будет', 'может', 'раз', 'тоже', 'иногда', 'со', 'он', 'что', 'ней', 'но', 'да', 'где', 'чего', 'сейчас', 'под', 'ничего', 'себе', 'про', 'вот', 'наконец', 'для', 'какой', 'на', 'будто', 'так', 'мы', 'уже', 'можно', 'лучше', 'тот', 'она', 'к', 'как', 'только', 'между', 'над', 'было', 'ли', 'то', 'там', 'себя', 'всю', 'вас', 'без', 'а', 'хоть', 'быть', 'ей', 'чем', 'один', 'ведь', 'ты', 'вы', 'теперь', 'меня', 'тут', 'зачем', 'больше', 'хорошо', 'конечно', 'ж', 'во', 'кто', 'им', 'всегда', 'все', 'нее', 'ему', 'надо', 'я', 'ним', 'уж', 'еще', 'если', 'они', 'куда', 'чтобы', 'через', 'о', 'из', 'чтоб', 'в', 'была', 'потому', 'были', 'том', 'более', 'вам', 'нас', 'разве', 'его', 'совсем', 'об', 'этой', 'эту', 'эти', 'и', 'потом', 'сам', 'ее', 'этот', 'есть', 'пере

In [16]:
import re
def lemmatize_filter(mstem_instance, text, stop_words):
    lemmatized = pd.Series([''.join(mstem_instance.lemmatize(text_row)) for text_row in text])
    filtered = []
    for row in lemmatized:
        filtered.append(' '.join([word for word in re.split(" |, |\n |:", row) if word and word not in stop_words]))
    return pd.Series(filtered)

In [17]:
all_lemmas = lemmatize_filter(m, news_df['title'], stop_words)

In [18]:
all_lemmas

0         полиция стамбул собираться штурмовать ночной к...
1         расстрел посетитель ночной клуб стамбул попада...
2         губернатор стамбул нападение ночной клуб погиб...
3         губернатор стамбул называть террористический а...
4         очевидец ночной клуб стамбул проходить беспреп...
5         турецкий клуб который атаковать террорист серь...
6         опубликовывать видео клуб "рейн" несколько час...
7            турецкий сми запрещать писать теракт стамбул\n
8         губернатор стамбул рассказывать нападение клуб...
9         опубликовывать видео начало атака ночной клуб ...
10        власть стамбул озвучивать официальный версия н...
11            сша небо сталкиваться легкомоторный самолет\n
12        руководство клуб "рейн" предупреждать подготов...
13        экс-голкипер ска бобровский становиться герой ...
14         четыре человек пострадать взрыв петарда москва\n
15        сми среди ранить ночной клуб стамбул иностранец\n
16        обама могерини столтенберг выр

In [19]:
news_df['title'].head()

0    Полиция Стамбула собирается штурмовать ночной ...
1    Расстрел посетителей в ночном клубе в Стамбуле...
2    Губернатор Стамбула: При нападении на ночной к...
3    Губернатор Стамбула назвал террористической ат...
4    Очевидцы: В ночной клуб в Стамбуле проходили б...
Name: title, dtype: object

In [20]:
news_df['title'] = all_lemmas

In [21]:
news_df['title'].head()

0    полиция стамбул собираться штурмовать ночной к...
1    расстрел посетитель ночной клуб стамбул попада...
2    губернатор стамбул нападение ночной клуб погиб...
3    губернатор стамбул называть террористический а...
4    очевидец ночной клуб стамбул проходить беспреп...
Name: title, dtype: object

In [22]:
# json parser
def json_parse(json_object, concat_str="\n"):
    text_from_json = [e["value"]["blocks"][0]["text"] for e in json_object if e["type"] == "TEXT" or e["type"] == "SUBTITLE"]
    return concat_str.join(text_from_json)

In [23]:
# html cleaner
def html_clean(html_string):
    from bs4 import BeautifulSoup
    return BeautifulSoup(html_string, "html.parser").get_text().replace(u'\xa0', u' ')

In [24]:
import math
def extract_text(raw_text):
    counter = 0
    extracted = []
    for row in raw_text:
        if row.__class__.__name__ == 'float':
            extracted.append("")
            continue
        j = get_json(row)
        counter += 1
        if j is None: # html
            extracted.append(html_clean(row))
        else: # json
            extracted.append(json_parse(j))
        pass
    return pd.Series(extracted)

In [25]:
try_ex = extract_text(news_df['content'])

/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:282: UserWarning: "https://life.ru/oskarvote/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:282: UserWarning: "https://projects.life.ru/apocalypse/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/dist-packages/bs4/__init__.py:282: UserWarning: "https://vk.com/videos-24199209?z=video-24199209_456242732%2Fclub24199209%2Fpl_-24199209_-2" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind 

In [26]:
news_df['content'] = try_ex

In [27]:
print(news_df[news_df['content'] != ""].shape)
print(news_df.shape)

(102205, 5)
(103327, 5)


In [28]:
# TODO: should work?
# news_df = news_df[news_df['content'] != ""]
# news_df = news_df[news_df['content'] != " "]

In [29]:
news_df.head()

,post_id,published_at,title,subtitle,content
0,1,2017-01-01T00:05:00Z,полиция стамбул собираться штурмовать ночной к...,"Согласно последним данным, в здании может нахо...",Полиция Стамбула собирается штурмовать ночной ...
1,2,2017-01-01T00:07:00Z,расстрел посетитель ночной клуб стамбул попада...,"Ранее сообщалось, что полицейские собираются ш...","Опубликовано видео, где злоумышленник расстрел..."
2,3,2017-01-01T00:21:00Z,губернатор стамбул нападение ночной клуб погиб...,"Ранее сообщалось, что от 40 до 60 человек полу...","Губернатор Стамбула Васип Шахин заявил, что пр..."
3,4,2017-01-01T00:29:00Z,губернатор стамбул называть террористический а...,"Согласно последним данным, погибло минимум 35 ...",Губернатор Стамбула Васип Шахин назвал террори...
4,5,2017-01-01T00:47:00Z,очевидец ночной клуб стамбул проходить беспреп...,Ранее губернатор Стамбула Васип Шахин назвал п...,"Посетители стамбульского клуба ""Рейна"", на кот..."


In [30]:
news_df.shape[0]

103327

In [31]:
news_df.to_csv('news_similarity_train_upd.csv', sep=',')

In [32]:
news_df.columns

Index(['post_id', 'published_at', 'title', 'subtitle', 'content'], dtype='object')

In [33]:
new_content = lemmatize_filter(m, news_df['content'], stop_words)

In [34]:
new_content

0         полиция стамбул собираться штурмовать ночной к...
1         опубликовывать видео злоумышленник расстрелива...
2         губернатор стамбул васип шахин заявлять нападе...
3         губернатор стамбул васип шахин называть террор...
4         посетитель стамбульский клуб "рейн" который со...
5         клуб "рейн" который сегодня совершать вооружат...
6         опубликовывать видео стамбульский клуб "рейн" ...
7         официальный власть турция экстренно издали нов...
8         слово губернатор стамбул васип шахин атака эли...
9         сеть появляться видео начало атака ночной клуб...
10        губернатор стамбул васип шахин рассказывать по...
11        американский город маккинни небо сталкиваться ...
12        хозяин ночной клуб "рейн" стамбул мехмет кочар...
13        бывший голкипер петербургский ска сергей бобро...
14        медучреждение подмосковье обращаться четыре че...
15        среди получать ранение клуб "рейн" стамбул гра...
16        президент сша барак обама генс

In [35]:
news_df['content'] = new_content

In [36]:
news_df.to_csv('news_similarity_train_upd.csv', sep=',')

In [37]:
sample_file = 'news_similarity_sample.csv'
sample_df = pd.read_csv(sample_file, sep=",", engine="c")

In [38]:
sample_df.columns

Index(['id', 'similarity'], dtype='object')

In [39]:
print(sample_df.head())
print(sample_df.columns)

            id  similarity
0  38277_31392         0.5
1  79621_65626         0.5
2  61134_17553         0.5
3  56944_64787         0.5
4   48365_7040         0.5
Index(['id', 'similarity'], dtype='object')


In [40]:
news_df['content'][0]

'полиция стамбул собираться штурмовать ночной клуб происходить нападение. согласно последний данные минимум злоумышленник находиться здание. ранить разный данные 40 60 человек.\nжертва становиться мало четыре человек. среди погибший — полицейский.\nво время празднество клуб "рейн" находиться 800 человек. неизвестный производить множество выстрел сообщаться взрывать граната.\n'

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
title_vect = CountVectorizer().fit(news_df['title'])
content_vect = CountVectorizer().fit(news_df['content'])
titles = title_vect.transform(news_df['title'])
contents = content_vect.transform(news_df['content'])

In [42]:
# news_df.loc[news_df['post_id'] == sample_df['id'][0].split("_")[0]]
news_df.loc[news_df['post_id'] == 38277]

,post_id,published_at,title,subtitle,content
38276,38277,2017-04-18T10:54:00Z,70% молодой водитель нравиться электромобиль\n,"Большая часть водителей, получающих права, не ...",итог опрос портал Driving-Tests большой часть ...


In [43]:
def get_index(df, post_id, col='post_id'):
    return df.index[df[col] == post_id][0]

In [44]:
# int(sample_df['id'][0].split("_")[0])

def elem(arr, index):
    return int(arr.split("_")[index])

In [45]:
get_index(news_df, 38277)

38276

In [46]:
news_df.loc[36512]

post_id                                                     36513
published_at                                 2017-04-12T15:39:00Z
title           экс-участница Little Big заявлять солистка-кар...
subtitle                                                      NaN
content         Little Big — самый популярный запад российский...
Name: 36512, dtype: object

In [63]:
from sklearn.decomposition import LatentDirichletAllocation
lda_titles = LatentDirichletAllocation(n_components=17, max_iter=17).fit_transform(old_titles)

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [64]:
lda_titles

array([[0.00653595, 0.00653595, 0.11764705, ..., 0.00653595, 0.45098039,
        0.22875817],
       [0.00735294, 0.00735294, 0.00735294, ..., 0.48599314, 0.13235294,
        0.27871274],
       [0.00653595, 0.00653595, 0.00653595, ..., 0.00653595, 0.11764706,
        0.22875817],
       ...,
       [0.00534759, 0.00534759, 0.00534759, ..., 0.00534759, 0.09625668,
        0.00534759],
       [0.1512605 , 0.00840336, 0.00840336, ..., 0.00840336, 0.00840336,
        0.1512605 ],
       [0.1512605 , 0.00840336, 0.00840336, ..., 0.29411764, 0.1512605 ,
        0.00840336]])

In [65]:
lda_contents = LatentDirichletAllocation(n_components=17, max_iter=17).fit_transform(old_contents)

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


In [50]:
lda_contents

array([[0.05454556, 0.00222232, 0.00222277, ..., 0.03355936, 0.08674995,
        0.00222302],
       [0.00208356, 0.00208354, 0.00208386, ..., 0.00208361, 0.07856683,
        0.00208392],
       [0.06019639, 0.00212817, 0.09335537, ..., 0.00212792, 0.21043396,
        0.05473594],
       ...,
       [0.0029417 , 0.00294162, 0.26427615, ..., 0.00294141, 0.24493511,
        0.00294158],
       [0.00188701, 0.0018871 , 0.00188729, ..., 0.00188725, 0.00188707,
        0.65735338],
       [0.00217462, 0.00217399, 0.00217412, ..., 0.07621625, 0.46025871,
        0.14463561]])

In [51]:
len(lda_contents), len(lda_titles)

(103327, 103327)

In [58]:
old_titles, old_contents

(<103327x36752 sparse matrix of type '<class 'numpy.int64'>'
 	with 745239 stored elements in Compressed Sparse Row format>,
 <103327x156878 sparse matrix of type '<class 'numpy.int64'>'
 	with 7941925 stored elements in Compressed Sparse Row format>)

<1x36752 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>

In [59]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import spatial
# float(cosine_similarity(titles[get_index(news_df, elem(sample_df['id'][i], 0))], titles[get_index(news_df, elem(sample_df['id'][i], 1))]))

In [66]:
title_similarities = []
content_similarities = []
total_similarity = []
def metric(s, elem1, elem2):
    if s == "sim":
        return cosine_similarity(elem1, elem2)
    if s == "dis":
        return 1 - spatial.distance.cosine(elem1, elem2)
    
    
for id_pair in sample_df['id']:
    first_index = get_index(news_df, elem(id_pair, 0))
    second_index = get_index(news_df, elem(id_pair, 1))
    title_sim = float(metric("dis", titles[first_index], titles[second_index]))
    content_sim = float(metric("dis", contents[first_index], contents[second_index]))
    title_weight = 0.5
    if title_sim != float(0) and content_sim != float(0):
        title_sim *= title_weight
        content_sim *= float(1) - title_weight
    title_similarities.append(title_sim)
    content_similarities.append(content_sim)
    total_similarity.append(title_sim + content_sim)
    pass

In [67]:
sample_df['similarity'] = pd.Series(total_similarity)
sample_df.head()

,id,similarity
0,38277_31392,0.622325
1,79621_65626,0.692452
2,61134_17553,0.087478
3,56944_64787,0.672185
4,48365_7040,0.746429


In [68]:
sample_df.to_csv('news_ag_submission11.csv', sep=',')